In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import random
from spellchecker import SpellChecker
import re
from typing import List
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
from collections import defaultdict
import TRUNAJOD.lexico_semantic_norms as lsn
import TRUNAJOD.surface_proxies as sp
import TRUNAJOD.semantic_measures as sm
import TRUNAJOD.ttr as ttr
import TRUNAJOD.givenness as gv
import TRUNAJOD.utils as utils
import TRUNAJOD.discourse_markers as dm
import TRUNAJOD.entity_grid as eg
import tarfile
import pickle
import spacy
import es_core_news_lg
import math
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [241]:
df = pd.read_excel("shortener_trunajod_indices.xlsx", index_col=0)
df.head()

,text,level,TTR lemma,TTR noun,TTR verb,TTR function,TTR content,TTR adj,TTR adv,TTR prp,...,avg_givenness_proj,min_givenness_proj,max_givenness_proj,kl_div,avg_dist_to_centroid,max_dist_to_centroid,min_dist_to_centroid,std_distance,relative_distance,det_dist
0,Era se una vez un pequeño escorpión a la orill...,1B,0.397163,0.636364,0.900000,0.556522,0.761905,0.888889,0.800000,0.705882,...,0.489610,0.280551,0.713717,0.250304,0.560975,0.729152,0.301215,14.183988,19.452718,4.588260e-12
1,"La cigüeña Aída Agüero, gran creadora de sombr...",1B,0.414847,0.902439,0.833333,0.452055,0.885714,0.894737,0.857143,0.714286,...,0.510210,0.382162,0.642056,0.233579,0.577674,0.760125,0.305093,14.367741,18.901802,-2.569041e-08
2,"En el cuartel, los bomberos están siempre de g...",1B,0.364621,0.733333,0.950000,0.452830,0.768116,1.000000,0.600000,0.470588,...,0.529138,0.389271,0.734457,0.290055,0.541887,0.708574,0.371756,14.569197,20.561280,-4.155671e-09
3,"Cierta vanidosa y acaudalada hormiga, preguntó...",1B,0.390863,0.692308,0.736842,0.531250,0.754386,1.000000,0.777778,0.750000,...,0.514968,0.360757,0.708050,0.199517,0.562314,0.698117,0.385223,14.367144,20.579861,-6.116545e-07
4,"Había una vez un niño que era muy pobre, pero ...",1B,0.369099,0.880000,0.730769,0.458333,0.840000,1.000000,0.500000,0.466667,...,0.507178,0.384525,0.654537,0.281055,0.567783,0.689033,0.369277,14.622747,21.222122,1.487500e-08


In [242]:
df_filtered = df.replace([np.inf, -np.inf], np.nan)
df_filtered = df_filtered.dropna()

In [495]:
level_map = {
    "1B": 0,
    "2B": 0,
    "3B": 0,
    "4B": 1,
    "5B": 1,
    "6B": 1,
    "7B": 2,
    "8B": 2,
}
df_filtered["group"] = df["level"].apply(lambda x: level_map[x])

In [496]:
df_train = df_filtered.drop(["level"], axis=1)
df_train.head()

,text,TTR lemma,TTR noun,TTR verb,TTR function,TTR content,TTR adj,TTR adv,TTR prp,TTR Diversidad léxica MTLD,...,min_givenness_proj,max_givenness_proj,kl_div,avg_dist_to_centroid,max_dist_to_centroid,min_dist_to_centroid,std_distance,relative_distance,det_dist,group
0,Era se una vez un pequeño escorpión a la orill...,0.397163,0.636364,0.900000,0.556522,0.761905,0.888889,0.800000,0.705882,65.757323,...,0.280551,0.713717,0.250304,0.560975,0.729152,0.301215,14.183988,19.452718,4.588260e-12,0
1,"La cigüeña Aída Agüero, gran creadora de sombr...",0.414847,0.902439,0.833333,0.452055,0.885714,0.894737,0.857143,0.714286,81.357096,...,0.382162,0.642056,0.233579,0.577674,0.760125,0.305093,14.367741,18.901802,-2.569041e-08,0
2,"En el cuartel, los bomberos están siempre de g...",0.364621,0.733333,0.950000,0.452830,0.768116,1.000000,0.600000,0.470588,53.274510,...,0.389271,0.734457,0.290055,0.541887,0.708574,0.371756,14.569197,20.561280,-4.155671e-09,0
3,"Cierta vanidosa y acaudalada hormiga, preguntó...",0.390863,0.692308,0.736842,0.531250,0.754386,1.000000,0.777778,0.750000,61.000000,...,0.360757,0.708050,0.199517,0.562314,0.698117,0.385223,14.367144,20.579861,-6.116545e-07,0
4,"Había una vez un niño que era muy pobre, pero ...",0.369099,0.880000,0.730769,0.458333,0.840000,1.000000,0.500000,0.466667,47.623596,...,0.384525,0.654537,0.281055,0.567783,0.689033,0.369277,14.622747,21.222122,1.487500e-08,0


# Split Data

We split the data into two groups. `X_train` will be used to train the models, `X_test` to evaluate them

In [497]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop(["group", "text"], axis=1),
    df_train.group, test_size=0.2)

In [565]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


selector = SelectKBest(f_classif, k=15)
selector.fit(X_train, y_train)
X_train_new = selector.transform(X_train)
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_columns = X_train.columns[selected_indices].tolist()
X_test_new = pd.DataFrame(selector.transform(X_test), columns=selected_columns)
selected_columns

['TTR lemma',
 'TTR function',
 'SM verb_syn_overlap',
 'SM lexical_overlap',
 'LSM concreteness',
 'LSM imageability',
 'LSM avg_context_availability',
 'SP Promedio Longitud Palabras letra',
 'SP Promedio Longitud Palabras sílaba',
 'SP Densidad de VERB|AUX',
 'r_distance',
 'approx_spell_errors',
 'avg_givenness_proj',
 'kl_div',
 'min_dist_to_centroid']

In [566]:
X_train_new = pd.DataFrame(X_train_new, columns=selected_columns)
X_train_new.head()

,TTR lemma,TTR function,SM verb_syn_overlap,SM lexical_overlap,LSM concreteness,LSM imageability,LSM avg_context_availability,SP Promedio Longitud Palabras letra,SP Promedio Longitud Palabras sílaba,SP Densidad de VERB|AUX,r_distance,approx_spell_errors,avg_givenness_proj,kl_div,min_dist_to_centroid
0,0.346863,0.290640,2.892857,0.424731,3.622500,4.027188,4.627813,4.915493,2.126761,0.095775,5.853458,74.0,0.478693,0.467646,0.247011
1,0.286432,0.313725,1.758621,0.446281,5.228462,5.893846,5.695385,4.191406,1.871094,0.128906,5.193512,40.0,0.480924,0.433080,0.360713
2,0.334034,0.339623,2.631579,0.443820,3.548571,3.827381,4.754048,4.698052,2.064935,0.074675,4.577560,61.0,0.462817,0.358478,0.122269
3,0.343699,0.359091,2.527273,0.381443,4.853571,5.816429,5.632857,4.563307,1.992248,0.162791,5.889199,83.0,0.457257,0.383632,0.251357
4,0.360248,0.376000,2.400000,0.362745,4.866667,5.568000,5.926667,4.311628,1.851163,0.120930,4.243224,49.0,0.496650,0.372178,0.291991


In [567]:
clusters = {}
for group in y_train:
    clusters[group] = X_train_new.values[y_train == group].mean(axis=0)
    clusters[group] /= np.linalg.norm(clusters[group])

In [568]:
for _, cluster in clusters.items():
    print(np.linalg.norm(cluster))

1.0
1.0
1.0


In [569]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train_new, y_train)

KNeighborsClassifier(n_neighbors=10)

In [570]:
y_pred_nn = neigh.predict(X_test_new)
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print("Accuracy: %.2f%%" % (accuracy_nn * 100.0))

Accuracy: 63.64%


In [571]:
X_test_norm = X_test_new / np.linalg.norm(X_test_new)

In [572]:
y_pred_trj_cluster = []
for X_sample in X_test_norm.values:
    min_dist = float("inf")
    pred_group = -1
    for cluster, group in clusters.items():
        dist = np.linalg.norm(X_sample - group)
        if dist < min_dist:
            pred_group = cluster
            min_dist = dist
    y_pred_trj_cluster.append(pred_group)

In [573]:
accuracy_trj = accuracy_score(y_test, y_pred_trj_cluster)
print("Accuracy: %.2f%%" % (accuracy_trj * 100.0))

Accuracy: 61.36%


In [574]:
xgb = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=None,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    nthread=4)
xgb.fit(X_train_new, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [575]:
print(y_train.value_counts(True))
print(y_test.value_counts(True))

1    0.421756
0    0.301527
2    0.276718
Name: group, dtype: float64
0    0.378788
1    0.371212
2    0.250000
Name: group, dtype: float64


In [544]:
y_pred = xgb.predict(X_test_new)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 59.85%


In [852]:
group_mapping = {
    0: "group_123B",
    1: "group_456B",
    2: "group_78B",
}

groups = df_train.group.apply(lambda x: group_mapping[x])
data = list(zip(df_train.text, groups, df_train[selected_columns].values))
train_portion = 0.8
train_idx = int(len(data) * train_portion)

random.shuffle(data)
train_data = data[0:train_idx]
test_data = data[train_idx:]
assert len(train_data) + len(test_data) == len(data)

In [591]:
import json


with open("train_data.jsonl", "w") as fp:
    for text, group in train_data:
        prompt = (
            "Clasifica el siguiente texto en uno de los siguientes estilos: group_123B, group_456B, group_78B.\n\n"
             f"Texto: {text}\n\n"
             "Estilo:")
        session = {"messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"{group}", "weight": 1}]}
        jsonl = json.dumps(session, ensure_ascii=False)
        fp.write(f"{jsonl}\n")
        
with open("test_data.jsonl", "w") as fp:
    for text, group in test_data:
        prompt = (
            "Clasifica el siguiente texto en uno de los siguientes estilos: group_123B, group_456B, group_78B.\n\n"
             f"Texto: {text}\n\n"
             "Estilo:")
        session = {"messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"{group}", "weight": 1}]}
        jsonl = json.dumps(session, ensure_ascii=False)
        fp.write(f"{jsonl}\n")

In [592]:
import openai
import os


openai_api_key = os.environ.get("OPENAI_API_KEY", "your key")
client = openai.OpenAI(api_key=openai_api_key)

In [593]:
openai.files.create(
  file=open("train_data.jsonl", "rb"),
  purpose='fine-tune'
)

FileObject(id='file-FDGf8KW5U2tPoPmt3FBxiu', bytes=1186413, created_at=1737236988, filename='train_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [594]:
job = client.fine_tuning.jobs.create(
    training_file="file-FDGf8KW5U2tPoPmt3FBxiu",
    model="gpt-4o-2024-08-06",
)
job

FineTuningJob(id='ftjob-xm48F5TkevqnIJcD0VFzyZKf', created_at=1737237062, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-JihYzTh2GjJjoPtZZ0kQdsbr', result_files=[], seed=1510144633, status='validating_files', trained_tokens=None, training_file='file-FDGf8KW5U2tPoPmt3FBxiu', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [836]:
def classify_text(text):
    prompt = (
            "Clasifica el siguiente texto en uno de los siguientes estilos: group_123B, group_456B, group_78B.\n\n"
             f"Texto: {text}\n\n"
             "Estilo:")
    
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:personal::ArBmNsh6",
        messages=[{"role": "user", "content": prompt}],
    )
    return response

In [855]:
len(train_data), len(test_data)

(524, 132)

In [857]:
groups = []
ft_class = []
trj_vals = []
for text, group, trj_val in tqdm(train_data, total=len(train_data)):
    response = classify_text(text)
    ft_class.append(response.choices[0].message.content)
    groups.append(group)
    trj_vals.append(trj_val)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 524/524 [05:17<00:00,  1.65it/s]


In [875]:
mapping = {
    "group_123B": "group_basic",
    "group_456B": "group_medium",
    "group_78B": "group_advanced",
}

norm_groups = list(map(lambda x: mapping[x], groups))
set(norm_groups), set(ft_class)

({'group_advanced', 'group_basic', 'group_medium'},
 {'**group_medium', 'group_advanced', 'group_basic', 'group_medium'})

In [871]:
cleaned_ft_class = []
for c in ft_class:
    for cc in list(set(norm_groups)):
        if cc in c:
            cleaned_ft_class.append(cc)
set(cleaned_ft_class)

{'group_advanced', 'group_basic', 'group_medium'}

In [867]:
sum([g in ft for ft, g in zip(ft_class, norm_groups)]) / len(ft_class)

0.8110687022900763

In [879]:
mapping_to_class = {
    "group_basic": 0,
    "group_medium": 1,
    "group_advanced": 2,
}

xgb_groups_train = list(map(lambda x: mapping_to_class[x], cleaned_ft_class))
df_train_ft = pd.DataFrame([t[2] for t in train_data], columns=selected_columns)
df_train_ft["gpt4_pred"] = xgb_groups_train
df_train_ft.head()

,TTR lemma,TTR function,SM verb_syn_overlap,SM lexical_overlap,LSM concreteness,LSM imageability,LSM avg_context_availability,SP Promedio Longitud Palabras letra,SP Promedio Longitud Palabras sílaba,SP Densidad de VERB|AUX,r_distance,approx_spell_errors,avg_givenness_proj,kl_div,min_dist_to_centroid,gpt4_pred
0,0.309689,0.294521,2.047619,0.382812,2.989524,3.393810,4.199524,5.666667,2.861582,0.073446,5.234709,79.0,0.457392,0.489791,0.338166,1
1,0.406699,0.523810,2.187500,0.311475,3.937500,4.162500,5.190000,4.433824,2.000000,0.132353,3.698460,31.0,0.514513,0.220831,0.370393,0
2,0.312178,0.295964,2.096774,0.360215,3.123704,2.784815,4.562963,4.479275,1.987047,0.119171,6.339204,61.0,0.478390,0.561491,0.272437,1
3,0.305128,0.270175,2.405405,0.366795,4.533437,5.111875,5.304687,4.545635,1.920635,0.085317,6.858405,132.0,0.457211,0.571003,0.275237,1
4,0.274956,0.282486,1.529412,0.225641,5.677742,6.160000,5.601613,4.174157,1.893258,0.146067,5.279514,70.0,0.447640,0.543874,0.315711,0


In [880]:
xgb.fit(df_train_ft, xgb_groups_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [881]:
test_groups = []
test_ft_class = []
test_trj_vals = []
for text, group, trj_val in tqdm(test_data, total=len(test_data)):
    response = classify_text(text)
    test_ft_class.append(response.choices[0].message.content)
    test_groups.append(group)
    test_trj_vals.append(test_trj_vals)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 132/132 [02:24<00:00,  1.09s/it]


In [898]:
len(test_groups), len(test_ft_class), len(test_trj_vals)

(132, 132, 0)

In [900]:
test_cleaned_ft_class = []
append_called = False
for c in test_ft_class:
    for cc in list(set(norm_groups)):
        if cc in c:
            test_cleaned_ft_class.append(cc)
            append_called = True
    if not append_called:
        test_cleaned_ft_class.append("group_basic")
    append_called = False
    
            
set(test_cleaned_ft_class), len(test_cleaned_ft_class)

({'group_advanced', 'group_basic', 'group_medium'}, 132)

In [901]:
test_norm_groups = list(map(lambda x: mapping[x], test_groups))

In [916]:
sum([g in ft for ft, g in zip(test_cleaned_ft_class, test_norm_groups)]) / len(ft_class)

0.18893129770992367

In [918]:
xgb_groups_test = list(map(lambda x: mapping_to_class[x], test_cleaned_ft_class))
df_test_ft = pd.DataFrame([t[2] for t in test_data], columns=selected_columns)
df_test_ft["gpt4_pred"] = xgb_groups_test
df_test_ft.head()

,TTR lemma,TTR function,SM verb_syn_overlap,SM lexical_overlap,LSM concreteness,LSM imageability,LSM avg_context_availability,SP Promedio Longitud Palabras letra,SP Promedio Longitud Palabras sílaba,SP Densidad de VERB|AUX,r_distance,approx_spell_errors,avg_givenness_proj,kl_div,min_dist_to_centroid,gpt4_pred
0,0.354486,0.380435,2.440000,0.354610,3.789565,4.036087,5.307391,4.818792,2.120805,0.110738,5.847132,66.0,0.503207,0.438893,0.262880,1
1,0.298153,0.334630,1.857143,0.372951,4.985556,5.562963,5.282222,4.539278,1.991507,0.148620,6.991246,105.0,0.451355,0.536036,0.229948,1
2,0.376900,0.484127,2.700000,0.413462,3.515714,3.868571,5.021429,5.108491,2.188679,0.127358,4.493285,60.0,0.477109,0.251999,0.295672,0
3,0.255875,0.240678,1.979592,0.523622,3.120682,3.244091,4.348636,4.842742,2.100806,0.114919,6.018451,71.0,0.420478,0.587276,0.349777,2
4,0.341040,0.417266,2.200000,0.333333,4.281429,4.965000,5.322857,4.233480,1.872247,0.096916,4.255356,33.0,0.467881,0.357677,0.243728,0


In [919]:
preds_ft_trj = xgb.predict(df_test_ft)
y_test_ft = list(map(lambda x: mapping_to_class[x], test_norm_groups))
accuracy_score(y_test_ft, preds_ft_trj)

0.75

In [927]:
df_train_trj = df_train_ft.drop(["gpt4_pred"], axis=1)
xgb2 = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=10,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    nthread=4)
xgb2.fit(df_train_trj, xgb_groups_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [928]:
df_test_trj = df_test_ft.drop(["gpt4_pred"], axis=1)
preds_trj = xgb2.predict(df_test_trj)

In [929]:
accuracy_score(y_test_ft, preds_trj)

0.6060606060606061